# CLASE 1: Fundamentos de autenticación y generación de JWT

## Objetivos de Aprendizaje

* Comprender el flujo de autenticación en sistemas reales.
* Implementar autenticación básica con tokens JWT en una API REST.
* Aplicar principios de seguridad y calidad de código al trabajar con credenciales y tokens.

---


## Estructura de la clase (4 horas)

| Hora | Tema                                                      |
| ---- | --------------------------------------------------------- |
| 1    | Introducción a JWT y fundamentos de autenticación         |
| 2    | Instalación, generación y verificación de tokens          |
| 3    | Simulación de login y validación de expiración            |
| 4    | Refactor de controladores y rutas protegidas              |
| 5    | Proyecto en clase: API con login + JWT                    |
| 6    | Validaciones, pruebas con errores y refuerzo de seguridad |

---


## Contenidos Detallados

### 1. ¿Qué es un token JWT y cómo funciona?


* **JWT (JSON Web Token)**: estándar abierto ([RFC 7519](https://datatracker.ietf.org/doc/html/rfc7519)) que define una forma compacta y segura de transmitir información entre partes como un objeto JSON.
* **Estructura**:
  `header.payload.signature` (codificados en Base64URL).
* **Ventajas**:

  * Stateless
  * Escalable
  * Seguro si se aplica correctamente
* **Uso común**:

  * Autenticación (Auth)
  * Autorización (Access Control)

> **Buenas prácticas**: Usar algoritmos robustos como HS256 o RS256. Evitar JWT sin expiración.

---


### 2. Instalación y uso de `python-jose` y `passlib`



pip install python-jose[cryptography] passlib[bcrypt] fastapi uvicorn pydantic

* **python-jose**: para crear/verificar tokens JWT
* **passlib**: para hash seguro de contraseñas


#### Función Básica para generar un token

In [ ]:
# Importa la librería 'jwt' del paquete 'python-jose',
# usada para codificar y decodificar tokens JWT (JSON Web Tokens)
from jose import jwt

# Importa utilidades de la librería estándar 'datetime':
# - datetime: para obtener la fecha y hora actuales
# - timedelta: para sumar o restar tiempo
# - timezone: para establecer zonas horarias (en este caso, UTC)
from datetime import datetime, timedelta, timezone

# Clave secreta que se usará para firmar el token.
# Esta clave debe mantenerse segura y no compartirse públicamente.
SECRET_KEY = "tu_clave_secreta"

# Algoritmo de firma utilizado para asegurar el token.
# HS256 es HMAC con SHA-256, un estándar seguro y comúnmente usado.
ALGORITHM = "HS256"

# Diccionario que representa el 'payload' del token JWT:
# - 'sub' (subject): identificador del usuario autenticado
# - 'exp' (expiration): momento exacto en que el token expira,
#   calculado como 30 minutos desde el momento actual en UTC
data = {
    "sub": "usuario123",  # Identificador del usuario (puede ser ID, username, etc.)
    "exp": datetime.now(timezone.utc) + timedelta(minutes=30)  # Expiración segura del token
}

# Genera un token JWT:
# - Incluye el 'payload' definido en 'data'
# - Lo firma usando la clave secreta (SECRET_KEY)
# - Utiliza el algoritmo de firma definido (HS256)
token = jwt.encode(data, SECRET_KEY, algorithm=ALGORITHM)

# Opcional
print("Token JWT generado:")
print(token)

---

### 3. Generación y verificación de tokens


#### Función reutilizable: Crear token:



In [ ]:
from jose import jwt
from datetime import datetime, timedelta, timezone

# Clave secreta utilizada para firmar el token JWT (debe mantenerse segura y privada)
SECRET_KEY = "tu_clave_secreta"

# Algoritmo de firma utilizado (HS256 es uno de los más comunes)
ALGORITHM = "HS256"

# Función para crear un token JWT
def crear_token(datos: dict, expiracion: int = 30):
    # Se crea una copia del diccionario de datos para no modificar el original
    to_encode = datos.copy()

    # Se calcula la fecha y hora de expiración del token (actual + minutos definidos)
    # Usamos datetime.now(timezone.utc) para que sea timezone-aware y evitar warnings
    expire = datetime.now(timezone.utc) + timedelta(minutes=expiracion)

    # Se agrega el campo 'exp' al diccionario, obligatorio para definir la expiración del token
    to_encode.update({"exp": expire})

    # Se genera el token JWT codificado y firmado con la clave y el algoritmo especificado
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)


# Datos de ejemplo para el token
datos_usuario = {"sub": "usuario123", "rol": "admin"}

# Crear token con datos de usuario
token_generado = crear_token(datos_usuario)

# Imprimir el token para mostrarlo
print("Token JWT generado:", token_generado)


#### Verificar token:



In [ ]:
from jose import jwt
from jose.exceptions import JWTError
from fastapi import HTTPException  # Solo si usas FastAPI

SECRET_KEY = "tu_clave_secreta"
ALGORITHM = "HS256"

def verificar_token(token: str):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        return payload
    except JWTError:
        # Si no usas FastAPI, cambia esta línea por algo como: raise ValueError("Token inválido o expirado")
        raise ValueError("Token inválido o expirado")

# Ejemplo de uso:
token_ejemplo = "aquí_va_un_token_valido"  # Debes reemplazar con un token válido

try:
    contenido = verificar_token(token_ejemplo)
    print("Token verificado con éxito:", contenido)
except Exception as e:
    print("Error al verificar token:", e)


### Código completo de Generación y Verificación de token

In [ ]:
from jose import jwt
from jose.exceptions import JWTError
from datetime import datetime, timedelta, timezone

# Si usas FastAPI, descomenta la siguiente línea y usa HTTPException para errores HTTP:
# from fastapi import HTTPException

# Clave secreta utilizada para firmar el token JWT (debe mantenerse segura y privada)
SECRET_KEY = "tu_clave_secreta"

# Algoritmo de firma utilizado (HS256 es uno de los más comunes)
ALGORITHM = "HS256"

# Función para crear un token JWT
def crear_token(datos: dict, expiracion: int = 30):
    to_encode = datos.copy()
    expire = datetime.now(timezone.utc) + timedelta(minutes=expiracion)
    to_encode.update({"exp": expire})
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)

# Función para verificar un token JWT
def verificar_token(token: str):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        return payload
    except JWTError:
        # Si usas FastAPI, lanza esta excepción para enviar un error HTTP 401:
        # raise HTTPException(status_code=401, detail="Token inválido o expirado")

        # Si no usas FastAPI, puedes lanzar una excepción estándar:
        raise ValueError("Token inválido o expirado")

# Datos de ejemplo para el token
datos_usuario = {"sub": "usuario123", "rol": "admin"}

# Crear token con datos de usuario
token_generado = crear_token(datos_usuario)

# Imprimir el token para mostrarlo
print("Token JWT generado:", token_generado)

# Verificar el token (ejemplo)
try:
    payload = verificar_token(token_generado)
    print("Token verificado, contenido:", payload)
except ValueError as e:
    print(str(e))


> **Buenas prácticas**:
>
> * Nunca expongas `SECRET_KEY` en el código. Usa variables de entorno.
> * Agrega validaciones explícitas para `exp`, `sub`, `aud`.

---


### 4. Simulación de login con usuario y contraseña

En un sistema real, cuando un usuario se registra o inicia sesión, nunca guardamos ni comparamos la contraseña en texto plano (es decir, tal cual la escribe el usuario), porque sería un riesgo enorme de seguridad si alguien accede a la base de datos.

### ¿Qué hacemos entonces?

Guardamos un **hash** de la contraseña, que es una transformación irreversible de la contraseña original. Así, aunque alguien acceda a la base de datos, no podrá ver las contraseñas reales.

Para hacer esto correctamente, usamos librerías especializadas como **Passlib** en Python.

---

### ¿Qué es Passlib?

**Passlib** es una librería de Python que facilita:

* Crear hashes seguros de contraseñas.
* Verificar contraseñas contra esos hashes.
* Usar algoritmos robustos y modernos (como bcrypt, argon2, pbkdf2\_sha256, etc.).
* Manejar sal (salt), que es un dato aleatorio que hace más seguro el hash.

---

### ¿Cómo funciona en el login?

1. Cuando un usuario se registra, su contraseña se convierte en un hash con Passlib y se guarda el hash (no la contraseña original).

2. Cuando un usuario intenta iniciar sesión, se toma la contraseña que escribe, se hashea y se compara con el hash almacenado.

3. Si coinciden, el login es exitoso.

---

### Ejemplo básico con Passlib (bcrypt):


In [ ]:
from passlib.context import CryptContext

# Crear contexto para hashing con bcrypt (uno de los algoritmos más seguros)
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

# Hash de la contraseña para guardar en base de datos (registro)
def hash_password(password: str) -> str:
    return pwd_context.hash(password)

# Verificar contraseña durante login
def verify_password(plain_password: str, hashed_password: str) -> bool:
    return pwd_context.verify(plain_password, hashed_password)

# Simulación:
password_original = "miClaveSuperSecreta123"
hashed = hash_password(password_original)
print("Hash generado:", hashed)

# Supongamos que el usuario ingresa esta contraseña para login
input_password = "miClaveSuperSecreta123"

# Verificar
if verify_password(input_password, hashed):
    print("¡Contraseña correcta! Login exitoso.")
else:
    print("Contraseña incorrecta.")


---

### ¿Por qué es importante?

* No guardas contraseñas en texto plano (nunca).
* Con Passlib evitas muchos errores comunes en hashing manual.
* Aumenta la seguridad general del sistema.
* Facilita futuras actualizaciones a algoritmos más seguros.

---


#### Login:



In [ ]:
from fastapi import FastAPI, Depends, HTTPException
from passlib.context import CryptContext
from jose import jwt, JWTError
from datetime import datetime, timedelta, timezone
from fastapi.security import OAuth2PasswordBearer
from pydantic import BaseModel

app = FastAPI()
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")
SECRET_KEY = "tu_clave_secreta"
ALGORITHM = "HS256"
ACCESS_TOKEN_EXPIRE_MINUTES = 30
usuarios = {}

def hashear_password(password: str) -> str:
    return pwd_context.hash(password)

def verificar_password(plain_password: str, hashed_password: str) -> bool:
    return pwd_context.verify(plain_password, hashed_password)

def crear_token(data: dict, expiracion: int = ACCESS_TOKEN_EXPIRE_MINUTES):
    to_encode = data.copy()  # Copia los datos para no modificar el original
    expire = datetime.now(timezone.utc) + timedelta(minutes=expiracion)  # Calcula fecha de expiración en UTC
    to_encode.update({"exp": expire})  # Agrega la expiración al payload
    # Codifica el token con la clave secreta y el algoritmo definido
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)

usuarios["admin"] = hashear_password("1234")

class LoginData(BaseModel):
    username: str
    password: str

@app.post("/login")
def login(datos: LoginData):
    user_pass = usuarios.get(datos.username)  # Obtiene la contraseña hasheada del usuario
    if not user_pass or not verificar_password(datos.password, user_pass):
        raise HTTPException(status_code=400, detail="Credenciales inválidas")
    
    token = crear_token({"sub": datos.username})
    
    return {"access_token": token, "token_type": "bearer"}


3. Ejecuta el servidor:

```bash
uvicorn main:app --reload
```

4. Prueba el endpoint `/login` con POST y un JSON:

```json
{
  "username": "admin",
  "password": "1234"
}
```

Y te devolverá un token JWT válido.



---

### 5. Sugerencias de estructura para login



#### Arquitectura recomendada:

```
app/
├── main.py
├── auth/
│   ├── auth_handler.py  # lógica JWT
│   ├── auth_service.py  # validación
├── routes/
│   └── user_routes.py
├── models/
│   └── user.py
```


> **Principios aplicados**:
>
> * SRP (Single Responsibility Principle)
> * DRY (Don't Repeat Yourself)
> * KISS (Keep It Simple, Stupid)

---


## **Proyecto por capas**

```
app/
├── main.py
├── core/
│   └── database.py          # conexión a PostgreSQL
├── auth/
│   ├── auth_handler.py      # lógica JWT
│   ├── auth_service.py      # hash/verify
├── routes/
│   └── user_routes.py       # rutas
├── models/
│   └── user.py              # modelo ORM
```

---


### Paso 1: `core/database.py`

In [ ]:
# Importamos las funciones necesarias desde SQLAlchemy y otras librerías
from sqlalchemy import create_engine  # Permite crear el motor de conexión a la base de datos
from sqlalchemy.ext.declarative import declarative_base  # Permite definir clases ORM que se traducen a tablas
from sqlalchemy.orm import sessionmaker  # Permite crear sesiones para interactuar con la base de datos
from sqlalchemy.exc import SQLAlchemyError  # Para capturar errores específicos de SQLAlchemy
import os  # Permite acceder a variables del sistema
from dotenv import load_dotenv  # Permite cargar variables desde un archivo .env al entorno del sistema

# Cargar las variables de entorno definidas en un archivo .env (si existe)
load_dotenv()

# Configuración de conexión con valores por defecto si no se encuentran en el entorno
DB_USER = os.getenv("DB_USER", "postgres")  # Usuario de la base de datos
DB_PASS = os.getenv("DB_PASS", "1126254560")  # Contraseña del usuario
DB_HOST = os.getenv("DB_HOST", "localhost")  # Dirección del host de la base de datos (localhost en desarrollo)
DB_PORT = os.getenv("DB_PORT", "5432")  # Puerto donde escucha PostgreSQL
DB_NAME = os.getenv("DB_NAME", "db_ejemplo")  # Nombre de la base de datos

# Construcción de la URL de conexión con el formato requerido por SQLAlchemy para PostgreSQL
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Intentamos crear el motor de conexión y las herramientas de SQLAlchemy
try:
    # Crea el motor de conexión a la base de datos usando la URL generada
    engine = create_engine(DATABASE_URL)

    # Crea la clase SessionLocal que servirá para crear sesiones de base de datos
    # autocommit=False: las transacciones no se confirman automáticamente
    # autoflush=False: los cambios no se sincronizan automáticamente
    # bind=engine: se vincula al motor creado
    SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

    # Clase base desde la cual se definirán las clases (modelos) ORM
    Base = declarative_base()

# En caso de error durante la conexión o creación de herramientas, se captura y lanza una excepción con mensaje claro
except SQLAlchemyError as e:
    raise Exception(f"Error al conectar con la base de datos: {str(e)}")

# Función que sirve como dependencia para obtener una sesión de base de datos
# Se usa, por ejemplo, en los endpoints de FastAPI para ejecutar consultas dentro de un contexto controlado
def get_db():
    db = SessionLocal()  # Se crea una nueva sesión a partir de SessionLocal
    try:
        yield db  # Se "entrega" la sesión al bloque que la necesite (por ejemplo, un endpoint)
    except SQLAlchemyError as e:
        db.rollback()  # Si ocurre un error, se hace rollback de cualquier cambio no confirmado
        raise Exception(f"Error en la operación de base de datos: {str(e)}")  # Se lanza una excepción personalizada
    finally:
        db.close()  # Siempre se cierra la sesión, haya error o no, para liberar recursos


---

### Paso 2: `models/user.py`



In [ ]:
# Importamos los tipos de columna que vamos a usar en la definición del modelo
from sqlalchemy import Column, Integer, String, DateTime

# Importamos una función de utilidad para obtener la hora actual del servidor
from sqlalchemy.sql import func

# Importamos la clase Base desde nuestro archivo de base de datos
# Esta clase es la que sirve como base para todos los modelos ORM
from core.database import Base

# Definimos el modelo de la tabla 'users', que representa a los usuarios del sistema
class User(Base):
    # Nombre de la tabla en la base de datos
    __tablename__ = "users"
    
    # Columna 'id': clave primaria (primary_key), tipo entero, con índice para búsquedas rápidas
    id = Column(Integer, primary_key=True, index=True)
    
    # Columna 'username': texto de hasta 50 caracteres, debe ser único, no puede ser nulo, con índice
    username = Column(String(50), unique=True, index=True, nullable=False)
    
    # Columna 'hashed_password': texto de hasta 255 caracteres, obligatorio
    # Aquí se guarda la contraseña cifrada del usuario, no la contraseña original
    hashed_password = Column(String(255), nullable=False)
    
    # Columna 'created_at': almacena la fecha y hora de creación del usuario
    # Se asigna automáticamente con la hora actual del servidor cuando se crea el registro
    created_at = Column(DateTime(timezone=True), server_default=func.now())
    
    # Columna 'updated_at': almacena la fecha y hora de la última modificación del usuario
    # Se actualiza automáticamente cada vez que se modifica el registro
    updated_at = Column(DateTime(timezone=True), onupdate=func.now())


---

### Paso 3: `auth/auth_service.py`



In [ ]:
# Importamos CryptContext desde la librería passlib, que nos permite manejar el hashing de contraseñas
from passlib.context import CryptContext

# Creamos un contexto de encriptación especificando que vamos a usar el algoritmo "bcrypt"
# "deprecated='auto'" significa que passlib elegirá automáticamente si un esquema está obsoleto
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

# Función para hashear (encriptar) una contraseña en texto plano
# Recibe como parámetro una contraseña tipo string
# Retorna la contraseña encriptada como string
def hashear_password(password: str) -> str:
    return pwd_context.hash(password)

# Función para verificar si una contraseña en texto plano coincide con su versión encriptada
# Recibe la contraseña original (sin cifrar) y la contraseña cifrada
# Retorna True si coinciden, False si no
def verificar_password(plain_password: str, hashed_password: str) -> bool:
    return pwd_context.verify(plain_password, hashed_password)


---

### Paso 4: `auth/auth_handler.py`



In [ ]:
# Importamos clases y funciones necesarias para manejar fechas y horas
from datetime import datetime, timedelta, timezone

# Importamos funciones para codificar y decodificar JWT desde la librería `python-jose`
from jose import jwt, JWTError

# Clave secreta usada para firmar y verificar los tokens JWT
SECRET_KEY = "clave_super_secreta"

# Algoritmo de cifrado que se usará para el token (HS256 = HMAC con SHA-256)
ALGORITHM = "HS256"

# Tiempo de expiración del token de acceso en minutos (30 minutos por defecto)
ACCESS_TOKEN_EXPIRE_MINUTES = 30

# Función para crear un token JWT
# Recibe un diccionario `data` con los datos que queremos codificar
# y un tiempo de expiración opcional en minutos (por defecto: 30)
def crear_token(data: dict, expiracion: int = ACCESS_TOKEN_EXPIRE_MINUTES):
    # Hacemos una copia del diccionario de datos para no modificar el original
    to_encode = data.copy()
    
    # Calculamos el tiempo de expiración sumando los minutos actuales
    expire = datetime.now(timezone.utc) + timedelta(minutes=expiracion)
    
    # Agregamos el campo 'exp' (expiración) al diccionario que se codificará
    to_encode.update({"exp": expire})
    
    # Codificamos el token JWT con la clave secreta y el algoritmo especificado
    return jwt.encode(to_encode, SECRET_KEY, algorithm=ALGORITHM)

# Función para verificar y decodificar un token JWT
# Recibe el token como string y retorna los datos (payload) si es válido
# Si hay error (por ejemplo, token inválido o expirado), retorna None
def verificar_token(token: str):
    try:
        # Decodificamos el token con la clave secreta y algoritmo
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        return payload
    except JWTError:
        # Si el token es inválido o ha expirado, retornamos None
        return None


---

### Paso 5: `routes/user_routes.py`



In [ ]:
# Importamos herramientas de FastAPI para crear rutas, dependencias e indicar errores HTTP
from fastapi import APIRouter, Depends, HTTPException

# Importamos Session para interactuar con la base de datos mediante SQLAlchemy
from sqlalchemy.orm import Session

# Importamos BaseModel para validar los datos de entrada con Pydantic
from pydantic import BaseModel

# Importamos la función para obtener la sesión de base de datos
from core.database import get_db

# Importamos el modelo de usuario
from models.user import User

# Importamos funciones para hashear y verificar contraseñas
from auth.auth_service import hashear_password, verificar_password

# Importamos función para crear el token JWT
from auth.auth_handler import crear_token

# Creamos un router de FastAPI para manejar las rutas relacionadas con el usuario
router = APIRouter()

# Definimos el esquema de datos que se espera tanto en login como en registro
class LoginData(BaseModel):
    username: str
    password: str

# Ruta para el inicio de sesión (login)
@router.post("/login")
def login(data: LoginData, db: Session = Depends(get_db)):
    # Buscamos el usuario por nombre de usuario en la base de datos
    user = db.query(User).filter(User.username == data.username).first()
    
    # Si no existe el usuario o la contraseña es incorrecta, lanzamos error 401 (no autorizado)
    if not user or not verificar_password(data.password, user.hashed_password):
        raise HTTPException(status_code=401, detail="Credenciales inválidas")

    # Si las credenciales son válidas, generamos un token JWT con el nombre de usuario
    token = crear_token({"sub": user.username})
    
    # Retornamos el token y el tipo de autenticación
    return {"access_token": token, "token_type": "bearer"}

# Ruta para registrar un nuevo usuario
@router.post("/register")
def register(data: LoginData, db: Session = Depends(get_db)):
    # Verificamos si ya existe un usuario con ese nombre
    user = db.query(User).filter(User.username == data.username).first()
    
    # Si ya existe, lanzamos un error 400 (petición inválida)
    if user:
        raise HTTPException(status_code=400, detail="Usuario ya existe")

    # Hasheamos la contraseña del nuevo usuario
    hashed = hashear_password(data.password)

    # Creamos una nueva instancia del modelo User
    nuevo_usuario = User(username=data.username, hashed_password=hashed)

    # Agregamos el nuevo usuario a la base de datos
    db.add(nuevo_usuario)
    db.commit()  # Confirmamos los cambios en la base de datos
    db.refresh(nuevo_usuario)  # Obtenemos la versión actualizada del usuario

    # Retornamos un mensaje de éxito
    return {"mensaje": "Usuario registrado exitosamente"}


---

### Paso 6: `main.py`



In [ ]:
from fastapi import FastAPI, HTTPException
from core.database import Base, engine
from routes.user_routes import router as user_router
import logging

logging.basicConfig(
    level=logging.INFO,  # Nivel de registro (INFO, DEBUG, ERROR, etc.)
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'  # Formato del mensaje
)
logger = logging.getLogger(__name__)  # Creamos un logger específico para este archivo

app = FastAPI()

try:
    Base.metadata.create_all(bind=engine)  # Ejecuta la creación de todas las tablas definidas en los modelos
    logger.info("Base de datos inicializada correctamente")  # Mensaje de éxito
except Exception as e:
    logger.error(f"Error al inicializar la base de datos: {str(e)}")  # Mensaje de error
    raise  # Lanza nuevamente la excepción para detener la aplicación si hay fallo

app.include_router(user_router, prefix="/api/v1")  # Todas las rutas estarán bajo /api/v1 (ej: /api/v1/login)


---

### Ejecutar el proyecto


1. Asegúrate de tener PostgreSQL encendido y una base de datos creada:

```bash
createdb tu_db
```

2. Instala dependencias:

```bash
pip install fastapi[all] sqlalchemy psycopg2-binary passlib[bcrypt] python-jose
```

3. Ejecuta el servidor:

```bash
uvicorn app.main:app --reload
```

4. Abre tu navegador:
   📍 [http://127.0.0.1:8000/docs](http://127.0.0.1:8000/docs)

---




### 6. Validación de lógica de expiración



* Usar `exp` como clave de control
* Simular token expirado
* Retornar mensaje claro y seguro



In [ ]:
try:
    datos = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
except ExpiredSignatureError:
    raise HTTPException(status_code=401, detail="Token expirado")


---

### 7. Refactor de controladores y rutas protegidas



#### Ejemplo con dependencia:



In [ ]:
from fastapi.security import OAuth2PasswordBearer
oauth2_scheme = OAuth2PasswordBearer(tokenUrl="login")

@app.get("/protegido")
def ruta_protegida(token: str = Depends(oauth2_scheme)):
    payload = verificar_token(token)
    return {"usuario": payload["sub"]}


> **Código limpio**: Evitar lógica repetida en rutas. Usar funciones reutilizables y dependencias.

---


## Proyecto en clase

**Objetivo**: Implementar autenticación básica con JWT en una API de usuarios.

### Requisitos mínimos:

* Endpoint `/login` con validación de usuario
* Endpoint `/me` protegido por token JWT
* Hash de contraseña
* Expiración de token a los 15 minutos


---

## Actividades adicionales (60 min)

### 1. Probar login con credenciales inválidas

* Simular contraseñas incorrectas
* Validar errores HTTP 401 o 400

### 2. Probar tokens vencidos

* Reducir la expiración a 5 segundos para pruebas

### 3. Revisión de lógica condicional

* Controlar que solo usuarios autenticados puedan acceder

### 4. Simulación de ataques comunes:

* **Token modificado**
* **Token sin firma**
* **Fuerza bruta**

> Refuerzo de seguridad:
>
> * Validar longitud mínima de contraseña
> * Evitar exposición de detalles técnicos
> * Usar HTTPS siempre
> * Implementar logout (invalidez del token en backend o blacklist)

---

## Normas internacionales aplicadas

| Norma             | Aplicación                                                                  |
| ----------------- | --------------------------------------------------------------------------- |
| **OWASP Top 10**  | Prevención de exposición de datos, validación de entrada, control de acceso |
| **PEP 8**         | Nombres descriptivos, organización de imports, espaciado                    |
| **PEP 20**        | Código legible y explícito                                                  |
| **ISO/IEC 27001** | Enfoque en confidencialidad e integridad de tokens                          |

---

## Recursos adicionales

* Documentación de JWT: [https://jwt.io/](https://jwt.io/)
* python-jose: [https://github.com/mpdavis/python-jose](https://github.com/mpdavis/python-jose)
* OWASP Cheatsheets: [https://cheatsheetseries.owasp.org/](https://cheatsheetseries.owasp.org/)

---

